# Simple Solana Top Traders Explorer

This notebook provides a simple way to browse the **gold layer** `top_traders` table in MinIO.

## Quick Setup
- **MinIO Endpoint**: localhost:9000 (internal: minio:9000)  
- **Credentials**: minioadmin / minioadmin123
- **Bucket**: solana-data
- **Table**: gold/top_traders

In [ ]:
# Debug: Check connection and file access
import duckdb

# Create connection for debugging
debug_conn = duckdb.connect()

print("🔍 Debug: Testing MinIO connection...")

try:
    # Install and load httpfs extension
    debug_conn.execute("INSTALL httpfs;")
    debug_conn.execute("LOAD httpfs;")
    
    # Configure MinIO credentials
    debug_conn.execute("SET s3_endpoint='localhost:9000';")
    debug_conn.execute("SET s3_access_key_id='minioadmin';")
    debug_conn.execute("SET s3_secret_access_key='minioadmin123';")
    debug_conn.execute("SET s3_use_ssl=false;")
    debug_conn.execute("SET s3_url_style='path';")
    
    print("✅ Extensions loaded and configured")
    
    # Test direct file access instead of listing
    print("\n🧪 Testing query patterns...")
    
    # Test single star pattern (should fail)
    try:
        single_star = debug_conn.execute("SELECT COUNT(*) FROM read_parquet('s3://solana-data/gold/top_traders/*.parquet')").fetchone()[0]
        print(f"✅ Single * pattern works: {single_star} records")
    except Exception as e:
        print(f"❌ Single * pattern failed: {str(e)[:100]}...")
    
    # Test double star pattern (should work for partitioned data)
    try:
        double_star = debug_conn.execute("SELECT COUNT(*) FROM read_parquet('s3://solana-data/gold/top_traders/**/*.parquet')").fetchone()[0]
        print(f"✅ Double ** pattern works: {double_star} records")
    except Exception as e:
        print(f"❌ Double ** pattern failed: {str(e)[:100]}...")
        
    # Test if any parquet files exist at all
    try:
        any_parquet = debug_conn.execute("SELECT COUNT(*) FROM read_parquet('s3://solana-data/**/*.parquet')").fetchone()[0]
        print(f"✅ Found total parquet files in bucket: {any_parquet}")
    except Exception as e:
        print(f"❌ No parquet files accessible: {str(e)[:100]}...")
        
except Exception as e:
    print(f"❌ Connection setup failed: {e}")
    print("\n💡 Troubleshooting steps:")
    print("1. Check if containers are running: docker-compose ps")
    print("2. Check MinIO service: docker-compose logs minio")
    print("3. Verify MinIO is accessible: curl http://localhost:9000/minio/health/live")

debug_conn.close()
print("\n" + "="*50)

In [ ]:
# Simple setup - just DuckDB
import duckdb

# Create connection
conn = duckdb.connect()

# Load S3 extension and configure for MinIO
conn.execute("INSTALL httpfs;")
conn.execute("LOAD httpfs;")

# Configure MinIO credentials
conn.execute("SET s3_endpoint='localhost:9000';")
conn.execute("SET s3_access_key_id='minioadmin';")
conn.execute("SET s3_secret_access_key='minioadmin123';")
conn.execute("SET s3_use_ssl=false;")
conn.execute("SET s3_url_style='path';")

print("✅ DuckDB connected to MinIO lakehouse!")

# Test connection by listing S3 buckets
try:
    buckets = conn.execute("SELECT * FROM duckdb_s3_ls('s3://')").df()
    print(f"Available buckets: {list(buckets['name'])}")
except Exception as e:
    print(f"Warning: Could not list buckets: {e}")

In [ ]:
# Browse top traders table
# Note: Data is partitioned by year/month/performance_tier
query = """
SELECT * 
FROM read_parquet('s3://solana-data/gold/top_traders/**/*.parquet')
LIMIT 10
"""

# Execute and display
result = conn.execute(query).df()
print(f"Found {len(result)} top traders (showing first 10)")
print(f"Columns: {list(result.columns)}")
result

## Top Traders Analysis

In [ ]:
# Get total count of top traders
count_query = """
SELECT COUNT(*) as total_traders
FROM read_parquet('s3://solana-data/gold/top_traders/**/*.parquet')
"""

total = conn.execute(count_query).fetchone()[0]
print(f"Total top traders in gold layer: {total}")

In [ ]:
# Top 5 traders by total PnL
top_pnl_query = """
SELECT 
    wallet_address,
    total_pnl_all,
    roi_all,
    trade_count_all,
    win_rate_all
FROM read_parquet('s3://solana-data/gold/top_traders/**/*.parquet')
ORDER BY total_pnl_all DESC
LIMIT 5
"""

top_traders = conn.execute(top_pnl_query).df()
print("🏆 Top 5 Traders by Total PnL:")
top_traders

In [ ]:
# Performance tiers distribution
tier_query = """
SELECT 
    performance_tier,
    COUNT(*) as trader_count,
    AVG(total_pnl_all) as avg_pnl,
    AVG(roi_all) as avg_roi,
    AVG(win_rate_all) as avg_win_rate
FROM read_parquet('s3://solana-data/gold/top_traders/**/*.parquet')
GROUP BY performance_tier
ORDER BY 
    CASE 
        WHEN performance_tier = 'elite' THEN 1
        WHEN performance_tier = 'strong' THEN 2
        WHEN performance_tier = 'promising' THEN 3
        ELSE 4
    END
"""

tiers = conn.execute(tier_query).df()
print("📊 Performance Tiers:")
tiers

In [ ]:
# Recent trader additions
recent_query = """
SELECT 
    wallet_address,
    first_seen_date,
    last_updated,
    total_pnl_week,
    roi_week,
    performance_tier
FROM read_parquet('s3://solana-data/gold/top_traders/**/*.parquet')
ORDER BY first_seen_date DESC
LIMIT 5
"""

recent = conn.execute(recent_query).df()
print("🆕 Most Recently Added Traders:")
recent

## Custom Queries

You can write your own SQL queries to explore the data:

In [ ]:
# Example: Find traders with high win rate and positive weekly PnL
custom_query = """
SELECT 
    wallet_address,
    win_rate_all,
    total_pnl_week,
    trade_count_week,
    performance_tier
FROM read_parquet('s3://solana-data/gold/top_traders/**/*.parquet')
WHERE win_rate_all > 0.6 
  AND total_pnl_week > 0
  AND trade_count_week > 0
ORDER BY total_pnl_week DESC
LIMIT 10
"""

high_performers = conn.execute(custom_query).df()
print("🎯 High Win Rate + Positive Weekly PnL:")
high_performers

In [ ]:
# Check the schema (all columns)
schema_query = """
DESCRIBE SELECT * FROM read_parquet('s3://solana-data/gold/top_traders/**/*.parquet')
"""

schema = conn.execute(schema_query).df()
print("📋 Full Schema (28 columns):")
for _, row in schema.iterrows():
    print(f"  {row['column_name']}: {row['column_type']}")

In [ ]:
# Close connection when done
conn.close()
print("Connection closed.")